# Notebook to play with EMAs

Latest version: 2024-08-20  
Author: MvS

## Description

Similar approach to SMA notebook but using the more volatile exponential moving average (EMA) indicator.

## Result

A vague trading signal (Long/Short) based on Low/High EMAs and a stop-loss estimation.


In [ ]:
import yfinance as yf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import datetime
from math import log2
from importlib import reload  # Python 3.4+
import utils.indicator_utils as idc

import logging
import sys
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s")

# Standard SMAs
periods = [200, 50]

# parameters of the EMA:
# window length
ema_length = 30
# widening factor
ema_scaler = 1.03

# stock = "FROG"
# stock = "TSLA"
# stock = "NVDA"
# stock = "SAP.TO"
# stock = "GLEN.L"
stock = "ADM.L"
stock = "SPX.L1234"

dt_end = datetime.datetime.today()
# Define real-time interval:
#  - assume to display at least the number of sample points of the larger period
#  - this requires double the number of points to create the averaging
#  - plus considering non-trading days - yfinance returns only trading days, howevers
dt_data_start = dt_end - datetime.timedelta(days=max(periods + [ema_length]) * 3)

try:
    # Grab sufficient stock data for averaging SMAs
    load_df = yf.download(
        f"{stock}",
        start=dt_data_start.strftime('%Y-%m-%d'),
        end=dt_end.strftime('%Y-%m-%d'),
        progress=False,
    )

    assert load_df.shape[1] == 6 and load_df.shape[0] > max(periods + [ema_length])

except AssertionError:
    logging.info(f"Download failed for symbol {stock}.  Skipping...")

### Define dynamic stop-losses based on EMAs

- use worst price quote of each day and calculate an exponential 30-day mean to define a stop-loss for long positions
- reciprocately, the best price quote of each day and calculate an exponential 30-day mean to define a stop-loss for short positions
- both curves are tracking each other with an offset and define a corridor of insignificant price action


In [ ]:
reload(idc)

sym_df = load_df.copy()

# Compute the simple moving average (SMA)
for period in periods:
    sym_df[f"SMA_{period:03}"] = idc.sma(sym_df['Close'], length=period)

long_ema_str = f"EMA_{ema_length:03}_Long"
short_ema_str = f"EMA_{ema_length:03}_Short"
min_sma_str = f"SMA_{min(periods):03}"
max_sma_str = f"SMA_{max(periods):03}"

# Compute two scaled EMAs based on daily Highs and Lows
# STOP LOSS LONG
sym_df[long_ema_str] = idc.ema(sym_df['Low'], length=ema_length) / ema_scaler

# STOP LOSS SHORT
sym_df[short_ema_str] = idc.ema(sym_df['High'], length=ema_length) * ema_scaler

# Now that we calculated the SMAs and EMAs, we can remove the data points before the actual AOI that we want.
sym_df = sym_df[-max(periods) :].copy()

# Add helpers for time period calculation
dt_start = sym_df.index[0]
sym_df['dt_start'] = dt_start
sym_df['dt_end'] = dt_end

# Add helper to scale signal strength
sig_scale = int(log2(sym_df[max_sma_str][-ema_length:].mean()))
sym_df['sig_scale'] = sig_scale

logging.info(
    f"""Calculating SMAs for {periods} length starting from:
  {dt_start.strftime('%Y-%m-%d')} and ending on {dt_end.strftime('%Y-%m-%d')}
  and containing {sym_df.shape} data points."""
)

logging.info(
    f"""Calculating EMAs for {ema_length} window length
  and containing a {(ema_scaler - 1)*100:4.1f} % scaler."""
)

# Define the columns and their styles
columns = [
    min_sma_str,
    max_sma_str,
    'High',
    'Low',
    long_ema_str,
    short_ema_str,
]
colors = ['blue', 'gold', 'darkgreen', 'red', 'forestgreen', 'orangered']
linestyles = ['-', '-', '-', '-', ':', ':']

# Plot each column separately
ax = None
for col, color, style in zip(columns, colors, linestyles):
    ax = sym_df[col].plot(color=color, linestyle=style, ax=ax)

### Using the breakout from the stop-loss corridor as a trading signal

- a trading day where the daily low has cleared the upper bound/short stop-loss can be a long/buy signal
- conversely, a trading day where the daily high has cleared the lower bound/long stop-loss can be a short/sell signal

In [ ]:
# Define the corridors for operation as strong deviations of fast from slow SMA
def valid_signal(x):
    high = f"High"
    low = f"Low"
    long = long_ema_str
    short = short_ema_str

    corridor_scaler = 1.00

    if x[low] > x[short] * corridor_scaler:
        return 1
    elif x[high] < x[long] / corridor_scaler:
        return -1
    else:
        return 0


# don't need loc unless working with view/slice of df
# sym_df.loc[:, 'Valid'] = sym_df.apply(valid_signal, axis=1)
sym_df['Valid'] = sym_df.apply(valid_signal, axis=1)

sym_df['Valid_scaled'] = 2**sig_scale + (sym_df['Valid'] * 2 ** (sig_scale - 2))

# Define the columns and their styles
columns = [
    "High",
    "Low",
    long_ema_str,
    short_ema_str,
    'Valid_scaled',
]
colors = ['darkgreen', 'red', 'forestgreen', 'orangered', 'grey']
linestyles = ['-', '-', ':', ':', '-']

# Plot each column separately
ax = None
for col, color, style in zip(columns, colors, linestyles):
    ax = sym_df[col].plot(color=color, linestyle=style, ax=ax)

In [ ]:
# Despiking a curve for single outliers


# Compare neighbor values: spike has two opposites on either side with same polarity
def despike(x):
    if x['Valid_N_Shift'] == x['Valid_P_Shift'] and x['Valid'] != x['Valid_N_Shift']:
        return x['Valid_N_Shift']
    elif pd.isna(x['Valid_P_Shift']):
        return 0.0
    else:
        return x['Valid']


# Shifting curve forward / backward
sym_df['Valid_P_Shift'] = sym_df['Valid'].shift(1)
sym_df['Valid_N_Shift'] = sym_df['Valid'].shift(-1)

sym_df['Valid_Despike'] = sym_df.apply(despike, axis=1)

sym_df[['Valid', 'Valid_Despike']].plot(color=['grey', 'red'])

# Replace old with new
sym_df['Valid'] = sym_df['Valid_Despike']

# Extra iterations
for iter in range(0, 3):
    # Shifting curve forward / backward
    sym_df['Valid_P_Shift'] = sym_df['Valid'].shift(1)
    sym_df['Valid_N_Shift'] = sym_df['Valid'].shift(-1)

    sym_df['Valid_Despike'] = sym_df.apply(despike, axis=1)

    # Replace old with new
    sym_df['Valid'] = sym_df['Valid_Despike']

sym_df['Valid_scaled'] = 2**sig_scale + (sym_df['Valid'] * 2 ** (sig_scale - 2))

# clean up
sym_df.drop(
    ['Valid_P_Shift', 'Valid_N_Shift', 'Valid_Despike'],
    axis=1,
    inplace=True,
    errors='ignore',
)

In [ ]:
def get_signal(x):
    if x['Valid'] != x['Shift'] and x['Valid'] != 0:
        return x['Valid']
    else:
        return 0


# Shift forward
sym_df['Shift'] = sym_df['Valid'].shift(1)
# Fill NaN
sym_df['Shift'] = sym_df['Shift'].bfill()

# Identify signal onsets
sym_df['On_Signal'] = sym_df.apply(get_signal, axis=1)
sym_df['On_Signal_scaled'] = 2**sig_scale + (
    sym_df['On_Signal'] * 2 ** (sig_scale - 2)
)

# Shift backward
sym_df['Shift'] = sym_df['Valid'].shift(-1)
# Fill NaN
sym_df['Shift'] = sym_df['Shift'].ffill()

# Identify signal terminations
sym_df['Off_Signal'] = sym_df.apply(get_signal, axis=1)
sym_df['Off_Signal_scaled'] = 2**sig_scale + (
    sym_df['Off_Signal'] * 2 ** (sig_scale - 2)
)

# Define the columns and their styles
columns = [
    "High",
    "Low",
    long_ema_str,
    short_ema_str,
    'Valid_scaled',
    'On_Signal_scaled',
    'Off_Signal_scaled',
]
colors = ['darkgreen', 'red', 'forestgreen', 'orangered', 'grey', 'red', 'black']
linestyles = ['-', '-', ':', ':', '-', ':', ':']

# Plot each column separately
ax = None
for col, color, style in zip(columns, colors, linestyles):
    ax = sym_df[col].plot(color=color, linestyle=style, ax=ax)

# clean up
sym_df.drop(
    ['Shift'],
    axis=1,
    inplace=True,
    errors='ignore',
)

In [ ]:
if sym_df['On_Signal'].any() != 0:
    logging.info('Some signals found...')

    # Filter rows where either 'On_Signal' or 'Off_Signal' is non-zero
    signals_df = sym_df[
        (sym_df['On_Signal'] != 0) | (sym_df['Off_Signal'] != 0)
    ].copy()

    # Calculate length of signals
    signals_df['Signal_len'] = -signals_df.index.to_series().diff(periods=-1).dt.days
    # Calculate age of signals
    signals_df['Signal_age'] = (
        signals_df['dt_end'] - signals_df.index.to_series()
    ).dt.days
    # Add missing length
    signals_df['Signal_len'] = signals_df['Signal_len'].fillna(signals_df['Signal_age'])

    # Calculate gap between signals
    signals_df['Signal_gap'] = signals_df.index.to_series().diff(periods=1).dt.days
    # Calculate ref days of signals
    signals_df['Signal_ref'] = (
        signals_df.index.to_series() - signals_df['dt_start']
    ).dt.days
    # Add missing gap
    signals_df['Signal_gap'] = signals_df['Signal_gap'].fillna(signals_df['Signal_ref'])

    logging.info(
        signals_df[
            [
                'Valid',
                'On_Signal',
                'Off_Signal',
                'Signal_ref',
                'Signal_len',
                'Signal_gap',
                'Signal_age',
            ]
        ].tail()
    )

In [ ]:
if sym_df['On_Signal'].any() != 0:
    # Add back the signal calculations
    sym_df = pd.concat(
        [
            sym_df,
            signals_df[['Signal_ref', 'Signal_len', 'Signal_gap', 'Signal_age']],
        ],
        axis=1,
    )
    # Fill non-signals
    sym_df.fillna(0, inplace=True)
else:
    sym_df.loc[:, ['Signal_ref', 'Signal_len', 'Signal_gap', 'Signal_age']] = 0

# # Type casting
sym_df = sym_df.astype(
    {
        'Valid': int,
        'Valid_scaled': int,
        'On_Signal': int,
        'On_Signal_scaled': int,
        'Off_Signal': int,
        'Off_Signal_scaled': int,
        'Signal_ref': int,
        'Signal_len': int,
        'Signal_gap': int,
        'Signal_age': int,
    }
)

sym_df.tail()
# sym_df.columns
# sym_df.dtypes

### Plotting the Chart

In [ ]:
import plotly.graph_objects as go

fig = go.Figure(
    data=[
        go.Candlestick(
            x=sym_df.index,
            open=sym_df['Open'],
            high=sym_df['High'],
            low=sym_df['Low'],
            close=sym_df['Close'],
            name=f"{stock}",
        ),
        go.Scatter(
            x=sym_df.index,
            y=sym_df[long_ema_str],
            mode='lines',
            name=long_ema_str,
            line=dict(color='#FF5252', width=2, dash='dot'),
        ),
        go.Scatter(
            x=sym_df.index,
            y=sym_df[short_ema_str],
            mode='lines',
            name=short_ema_str,
            line=dict(color='#4CAF50', width=2, dash='dot'),
        ),
        go.Scatter(
            x=sym_df.index,
            y=sym_df[f"Valid_scaled"],
            mode='lines',
            name=f"Validity",
            line=dict(color='#000000', width=2, dash='solid'),
        ),
        go.Scatter(
            x=sym_df.index,
            y=sym_df[f"On_Signal_scaled"],
            mode='lines',
            name=f"Signal",
            line=dict(color='#2196F3', width=2, dash='solid'),
        ),
    ]
)

fig.update_yaxes(type='log')

fig.update_layout(
    title='Standard candlesticks and trading signals based on EMAs',
    yaxis_title=f"{stock} Stock",
    xaxis_rangeslider_visible=False,
    width=1200,
    height=800,
)

fig.show()